## Importamos las librerias a utilizar

In [1]:
import pandas as pd
import json
pd.set_option('display.max_columns', None)

### Extraemos los datos de reviews

In [2]:
df = []
with open("../Datasets/review.json", "r",encoding="utf-8") as reviews:
    for review in reviews:
        register = json.loads(review)
        df.append(register)


In [3]:
reviews_df = pd.DataFrame(df)
reviews_df.shape

(6990280, 9)

- Lo transformamos a ***parquet***

In [4]:
reviews_df.to_parquet("../Datasets/reviews.parquet",engine="pyarrow")

In [2]:
# Ya que el archivo se va modificando dia a dia, se importa de nuevo, pero esta vez en formato parquet
reviews_p = pd.read_parquet("../../Datasets/reviews.parquet")

In [3]:
# Importamos el archivo business que contiene los negocios registrados en Yelp
business = pd.read_parquet("../../Datasets/Yelp_business.parquet") # Datos obtenidos del archivo EDA/business_yelp

In [32]:
business.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,California,93101,34.426679,-119.711197,5.0,7,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,Missouri,63123,38.551126,-90.335695,3.0,15,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Shipping Centers, Local Services, Notaries, Ma...","{'Friday': '8:0-18:30', 'Monday': '0:0-0:0', '..."


In [4]:
reviews_p.head(2)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18


- Filtramos los negocios que se encuentran dentro de los estados que nos interesan

In [5]:
states = ["Louisiana","Indiana"]
business_states = business[business["state"].isin(states)] 

- Realizamos un *`merge`* entre los *Dataframes* **business_states** y **reviews** utilizando la columna en comun *business_id*.

In [6]:
business_reviews = business_states.merge(reviews_p,on="business_id",how="inner")

In [15]:
business_reviews.isna().sum()

business_id         0
name                0
address             0
city                0
state               0
postal_code         0
latitude            0
longitude           0
stars_x             0
review_count        0
is_open             0
attributes      25532
categories         85
hours           65281
review_id           0
user_id             0
stars_y             0
useful              0
funny               0
cool                0
text                0
date                0
dtype: int64

In [7]:
business_reviews.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,is_open,attributes,categories,hours,review_id,user_id,stars_y,useful,funny,cool,text,date
0,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,Indiana,46227,39.637133,-86.127217,2.5,28,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","American (Traditional), Restaurants, Diners, B...","{'Friday': '6:0-22:0', 'Monday': '6:0-22:0', '...",YByDh56Hl11HoYdBm-uArA,9hhRs_n85m-jsKOXp3jt7Q,1.0,0,0,0,Went there at 4am and there was only one waitr...,2016-05-08 08:49:25
1,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,Indiana,46227,39.637133,-86.127217,2.5,28,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","American (Traditional), Restaurants, Diners, B...","{'Friday': '6:0-22:0', 'Monday': '6:0-22:0', '...",3vwdkRRoUe4PewzD8HvQbQ,-BX5x3Ys-2JzFba3LiWlHA,1.0,0,0,0,I had an immensely frustrating experience here...,2017-08-30 10:52:52


- Renombre de algunas columnas

In [9]:
business_reviews.rename(columns={"stars_x":"stars_mean",
                                 "stars_y" :"stars_review"},inplace=True)

- Eliminamos columnas que no seran utilizadas

In [10]:
business_reviews.drop(columns=["hours","attributes"],inplace=True)

- Exportamos a parquet

In [12]:
business_reviews.to_parquet("../../Datasets/business_reviews.parquet",engine="pyarrow")

In [17]:
business_reviews.fillna({"categories":""},inplace=True)

In [39]:
place = business_reviews[business_reviews["categories"].str.contains("Hotel")]
place.shape

(82826, 22)

In [42]:
place["name"].value_counts()

name
Louis Armstrong New Orleans International Airport  MSY    1850
The National WWII Museum                                  1650
French Quarter Phantoms                                   1499
Haunted History Tours                                     1314
Restaurant R'evolution                                    1231
                                                          ... 
Hyatt Place Indianapolis / Carmel                            5
Indy JoyRides                                                5
House of the Rising Sun B&B                                  5
Riverboat Travel Park                                        5
Claiborne Mansion                                            5
Name: count, Length: 1119, dtype: int64